In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.
The file _gams_py_gdb4.gdx is still active and was not deleted.
The file _gams_py_gdb5.gdx is still active and was not deleted.
The file _gams_py_gdb6.gdx is still active and was not deleted.


*Load specific modules used here:*

In [2]:
os.chdir(main)
%run stdPlotting.ipynb
os.chdir(d['py'])
import mOther

# Other modules

Most of these modules cannot actually run partially. Thus, the following simply showcase a bit of syntax for initialization.

*Load test data:*

In [3]:
t0 = 2019
name = 'TestModelData' #
db = GpyDB(os.path.join(d['data'], name), name = 'test', ws = d['work'])
ws = db.ws

In [4]:
# AggDB.subsetDB(db, db('s_HH'))

For variables that are defined over $t$, but where we do not yet have an initial value for all $t$, extrapolate from data:

*Note: This forces extrapolation of all variables defined over $t$ - if it is important that some variables are not extrapolated, they should be removed from this statement.*

In [5]:
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

## 1. ```IndexFund```

*This can be added to a model with production (and/or investment) modules that compute fundamental values.*

In [6]:
db['sIndexFund'] = db('s_p').union(db('s_i')) # assume all production and investment sectors are included in the index fund.
db_IDX = db.copy()
db_i = db_IDX
AggDB.subsetDB(db_IDX, db('sIndexFund'))

Initialize as such:

In [7]:
m_IdxFund = mOther.IndexFund('idxF', db_i, 'sIndexFund') # name of module, database, name of symbol that covers sectors included in the fund
m_i = m_IdxFund
m_i.initStuff()

Test write methods:

In [8]:
print(m_i.equationText)


$BLOCK B_idxF
	E_idxF[t]$(txE[t])..	vIdxFund[t] =E= sum(s$(sIndexFund[s]), uPortfolio[t,s]*vA[t,s]);
$ENDBLOCK



## 2. ```InventoryAR```

In [9]:
db['sInventory'] = pd.Index(['itory'], name = 's') # name of inventory investment sector
db_Invt = db.copy()
db_i = db_Invt
AggDB.subsetDB(db_i, db('sInventory'))

Initialize as such:

In [10]:
m_Invt = mOther.InventoryAR('InvtAR', db_i, 'sInventory') # name of module, database, name of symbol that covers sectors included in the fund
m_i = m_Invt
m_i.initStuff()

Test write methods:

In [11]:
print(m_i.equationText)


$BLOCK B_InvtAR
	E_InvtAR[t,s,n]$(dInventory[s,n] and tx0E[t])..	qD[t,s,n] =E= inventoryAR[s,n] * qD[t-1,s,n]/(1+g_LR);
$ENDBLOCK



This can actually run without any more fuss:

In [12]:
m_i.solve();

## 3. ```SmallOpenEq```

Initialization:

In [13]:
m_equi = mOther.SmallOpenEq('eq', db) # name of module, database
m_i = m_equi
m_i.initStuff()

Test write methods:

In [14]:
print(m_i.equationText)


$BLOCK B_eq_baseline
	E_eq_equi[t,n]$(nEqui[n] and txE[t])..	 sum(s$(d_qS[s,n]), qS[t,s,n]) =E= sum(s$(d_qD[s,n]), qD[t,s,n]);
$ENDBLOCK

$BLOCK B_eq_calib
	E_eq_equi_tx0E[t,n]$(nEqui[n] and tx0E[t])..	 sum(s$(d_qS[s,n]), qS[t,s,n]) =E= sum(s$(d_qD[s,n]), qD[t,s,n]);
$ENDBLOCK

